## Building Tools for LLM 🛠️

<img src="./Images/Tools.png" width="800" height="600" style="display: block; margin: auto;">

In [1]:
from dotenv import load_dotenv
from langchain_ollama import ChatOllama

load = load_dotenv('./../.env')


llm = ChatOllama(
    base_url="http://localhost:11434",
    model = "qwen2.5:latest",
    temperature=0.5,
    max_tokens = 250
)

### Installing a community

In [2]:
!pip install --upgrade --quiet  wikipedia


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [3]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [4]:
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

tool_response = wikipedia.invoke("Give me details of Avatar movie")

print(tool_response)

Page: Avatar (franchise)
Summary: Avatar is an American epic science fiction media franchise created by James Cameron, which began with the eponymous 2009 film. Produced by 20th Century Studios and distributed by Lightstorm Entertainment, it consists of associated merchandise, video games, and theme park attractions. Avatar is set in the mid-22nd century on Pandora, a lush habitable moon of a gas giant in the Alpha Centauri star system. The films' central conflict is between the indigenous Na'vi led by Jake Sully and Neytiri, and humans led by Colonel Miles Quaritch from the Resources Development Administration (RDA), a megacorp which has arrived on Pandora to colonize and pillage it for its natural resources. The title of the series refers to the genetically engineered Na'vi body operated from the brain that humans pilot to interact with on Pandora.
The first installment, Avatar, was released on December 18, 2009, and is the highest grossing motion picture of all-time when ticket pric

### Creating Custom Tools 

In [5]:
from langchain.tools import tool

@tool
def add_numbers(a: int, b: int) -> int:
    "Add two numbers and return results."
    return int(a) + int(b)

@tool
def subtract_numbers(a: int, b: int) -> int:
    "Subtract two numbers and return results."
    return int(a) - int(b)

@tool
def multiply_numbers(a: int, b: int) -> int:
    "Multiply two numbers and return results."
    return int(a) * int(b)

print(add_numbers.invoke({"a": 10, "b": 20}))

30


In [6]:
tools = [wikipedia, add_numbers, subtract_numbers, multiply_numbers]

list_of_tools = {tool.name: tool for tool in tools}

list_of_tools

# response = llm.invoke("When did avatar movie released the ways of water?")

# print(response.content)

llm_with_tools = llm.bind_tools(tools=tools)

tool_call_response = llm_with_tools.invoke("What is the double of 2")

print(tool_call_response.tool_calls)



[{'name': 'multiply_numbers', 'args': {'a': 2, 'b': 2}, 'id': '71de6e56-870e-4da6-a3ad-8004b71f57a7', 'type': 'tool_call'}]


### Execute the custom tools from LLM

In [7]:
from langchain_core.messages import HumanMessage

query = "What is the double of 2"

message = [HumanMessage(query)]


ai_message = llm_with_tools.invoke(query)

message.append(ai_message)

message

[HumanMessage(content='What is the double of 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:latest', 'created_at': '2025-02-18T21:07:45.230058Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1220774083, 'load_duration': 11453000, 'prompt_eval_count': 371, 'prompt_eval_duration': 24000000, 'eval_count': 55, 'eval_duration': 1182000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-01fcaae8-8799-45a2-aff3-cbe4843a7c25-0', tool_calls=[{'name': 'multiply_numbers', 'args': {'a': 2, 'b': 2}, 'id': '757b9b05-036c-49d9-8701-56e71e96614b', 'type': 'tool_call'}], usage_metadata={'input_tokens': 371, 'output_tokens': 55, 'total_tokens': 426})]

In [8]:
for tool_call in ai_message.tool_calls:
    tool_name = tool_call['name'].lower()
    
    execute_tool = list_of_tools[tool_name]
    
    tool_invoke = execute_tool.invoke(tool_call)
    
    message.append(tool_invoke)
    
message

[HumanMessage(content='What is the double of 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:latest', 'created_at': '2025-02-18T21:07:45.230058Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1220774083, 'load_duration': 11453000, 'prompt_eval_count': 371, 'prompt_eval_duration': 24000000, 'eval_count': 55, 'eval_duration': 1182000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-01fcaae8-8799-45a2-aff3-cbe4843a7c25-0', tool_calls=[{'name': 'multiply_numbers', 'args': {'a': 2, 'b': 2}, 'id': '757b9b05-036c-49d9-8701-56e71e96614b', 'type': 'tool_call'}], usage_metadata={'input_tokens': 371, 'output_tokens': 55, 'total_tokens': 426}),
 ToolMessage(content='4', name='multiply_numbers', tool_call_id='757b9b05-036c-49d9-8701-56e71e96614b')]

In [9]:
final_output = llm_with_tools.invoke(message)

print(final_output.content)

The double of 2 is 4.
